In [1]:
from pathlib import Path
import sys
import json
import pymongo
from flask import Flask
sys.path.append("gtex-pipeline/gene_model")
from collapse_annotation import collapse_annotation, Annotation

In [2]:
from pymongo import MongoClient
_client = MongoClient('localhost', 27017)
_db = _client.gandallab

In [3]:
from pymongo import InsertOne
from pymongo.errors import BulkWriteError
from pprint import pprint

def attr_str_to_dict(s):
    d = {}
    for item in s.split(';')[:-1]:
        item = item.strip().split(' ', maxsplit=1)
        d[item[0]] = item[1].strip('"')
    return d

def parse_gtf_line(line):
    line = line.rstrip()
    cols = line.split('\t')
    feature = {
        'chromosome': cols[0],
        'source': cols[1],
        'feature_type': cols[2],
        'start': int(cols[3]),
        'end': int(cols[4]),
        'score': cols[5],
        'strand': cols[6],
        'frame': cols[7]
    }
    feature.update(attr_str_to_dict(cols[8]))
    return feature

def mongo_bulk_write_genes(ops, count=0):
    try:
        _db.genes.bulk_write(ops, ordered=False)
    except BulkWriteError as bwe:
        pprint(bwe.details)
    print('\twrote', count+len(ops), 'genes to db')
    return len(ops)

def mongo_insert_gtf(f, name):
    if _db.genes.find_one({'file': name}, {}):
        print('\tGTF already inserted')
        return
    
    tx_count = 0
    gene_count = 0
    ops = []
    current_gene = None

    for line in f:
        if line[0] == '#': continue

        feat = parse_gtf_line(line)

        if feat['feature_type'] == 'gene':
            if current_gene:
                ops.append(InsertOne({
                    'file': name,
                    'gene_id_short': current_gene['gene_id'].rsplit('.', maxsplit=1)[0],
                    'gene': current_gene
                }))
            current_gene = feat
            current_gene['transcripts'] = []
        elif feat['feature_type'] == 'transcript':
            current_gene['transcripts'].append(feat)
            current_gene['transcripts'][-1]['exons'] = []
            tx_count += 1
            if tx_count % 1000 == 0:
                print('\tread', tx_count, 'transcripts from file')
        elif feat['feature_type'] == 'exon':
            current_gene['transcripts'][-1]['exons'].append(feat)
        
        if len(ops) >= 1000:
            gene_count += mongo_bulk_write_genes(ops, gene_count)
            ops = []
    
    if tx_count % 1000 != 0:
        print('\tread', tx_count, 'transcripts from file')
    if current_gene:
        ops.append(InsertOne({
            'file': name,
            'gene_id_short': current_gene['gene_id'].rsplit('.', maxsplit=1)[0],
            'gene': current_gene
        }))
    if ops:
        mongo_bulk_write_genes(ops, gene_count)

In [110]:
gtfs_path = Path('/mnt/z/Gandal/GTF')
for gtf in gtfs_path.glob('gencode*.gtf'):
    print('Processing', gtf.name)
    with open(gtf) as f:
        mongo_insert_gtf(f, gtf.stem)


Processing gencode.v35lift37.annotation.gtf
	read 1000 transcripts from file
	read 2000 transcripts from file
	read 3000 transcripts from file
	wrote 1000 genes to db
	read 4000 transcripts from file
	read 5000 transcripts from file
	read 6000 transcripts from file
	read 7000 transcripts from file
	wrote 2000 genes to db
	read 8000 transcripts from file
	read 9000 transcripts from file
	read 10000 transcripts from file
	read 11000 transcripts from file
	wrote 3000 genes to db
	read 12000 transcripts from file
	read 13000 transcripts from file
	read 14000 transcripts from file
	wrote 4000 genes to db
	read 15000 transcripts from file
	read 16000 transcripts from file
	read 17000 transcripts from file
	read 18000 transcripts from file
	wrote 5000 genes to db
	read 19000 transcripts from file
	read 20000 transcripts from file
	read 21000 transcripts from file
	wrote 6000 genes to db
	read 22000 transcripts from file
	read 23000 transcripts from file
	read 24000 transcripts from file
	read

In [25]:
_db.genes.drop_index('file_1_gene.gene_id_1')
#_db.genes.create_index([('file', pymongo.ASCENDING), ('gene.gene_name', pymongo.ASCENDING)])
_db.genes.create_index([('file', pymongo.ASCENDING), ('gene_id_short', pymongo.ASCENDING)])
#_db.genes.create_index('gene.gene_name')
#_db.genes.create_index('gene_id_short')
#_db.genes.create_index('gene.transcripts.transcript_id')
list(_db.genes.list_indexes())

[SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')]),
 SON([('v', 2), ('key', SON([('file', 1), ('gene.gene_name', 1)])), ('name', 'file_1_gene.gene_name_1')]),
 SON([('v', 2), ('key', SON([('gene.gene_name', 1)])), ('name', 'gene.gene_name_1')]),
 SON([('v', 2), ('key', SON([('gene.transcripts.transcript_id', 1)])), ('name', 'gene.transcripts.transcript_id_1')]),
 SON([('v', 2), ('key', SON([('gene_id_short', 1)])), ('name', 'gene_id_short_1')]),
 SON([('v', 2), ('key', SON([('file', 1), ('gene_id_short', 1)])), ('name', 'file_1_gene_id_short_1')])]

In [109]:
_db.genes.remove({'file': 'gencode.v35lift37.annotation'})

{'n': 62475, 'ok': 1.0}

In [19]:
# add missing gene_id_short fields - this should not be needed to be run again
count = 0
for gene in _db.genes.find({}, {'gene_id_short': 1, 'gene.gene_id': 1}):
    if 'gene_id_short' not in gene:
        gene_id_short = gene['gene']['gene_id'].rsplit('.', maxsplit=1)[0]
        _db.genes.update_one({'_id': gene['_id']}, {'$set': {'gene_id_short': gene_id_short}})
    count += 1
    if count % 1000 == 0:
        print('processed', count, 'genes')

processed 1000 genes
processed 2000 genes
processed 3000 genes
processed 4000 genes
processed 5000 genes
processed 6000 genes
processed 7000 genes
processed 8000 genes
processed 9000 genes
processed 10000 genes
processed 11000 genes
processed 12000 genes
processed 13000 genes
processed 14000 genes
processed 15000 genes
processed 16000 genes
processed 17000 genes
processed 18000 genes
processed 19000 genes
processed 20000 genes
processed 21000 genes
processed 22000 genes
processed 23000 genes
processed 24000 genes
processed 25000 genes
processed 26000 genes
processed 27000 genes
processed 28000 genes
processed 29000 genes
processed 30000 genes
processed 31000 genes
processed 32000 genes
processed 33000 genes
processed 34000 genes
processed 35000 genes
processed 36000 genes
processed 37000 genes
processed 38000 genes
processed 39000 genes
processed 40000 genes
processed 41000 genes
processed 42000 genes
processed 43000 genes
processed 44000 genes
processed 45000 genes
processed 46000 gen

In [20]:
_db.genes.count({'gene_id_short': None})

0

In [33]:
print(_db.genes.count())
print(len(_db.genes.distinct('gene.gene_name')))
print(len(_db.genes.distinct('gene_id_short')))
print(len(list(_db.genes.aggregate([{'$group': {'_id': {'gene_name': "$gene.gene_name", 'gene_id': "$gene_id_short"}}}]))))

78156
60225
62930
63116


In [72]:
def generate_selectize(f, gene_file=None):
    filter = {'file': gene_file} if gene_file else {}
    selectize = sorted(_db.genes.distinct('gene.gene_name', filter))
    f.write('function getSelectizeOptions(){ return ')
    json.dump([{'v': x} for x in selectize], f)
    f.write('; }')
    if hasattr(f, 'name'):
        print('Wrote', len(selectize), 'gene symbols to', f.name)

In [75]:
print(_db.genes.count({'file': 'All_unfiltered_128k_multiexon_A0.75_minRead3_talon'}))
print(len(_db.genes.distinct('gene_id_short', {'file': 'All_unfiltered_128k_multiexon_A0.75_minRead3_talon'})))


15681
15681


In [66]:
g = list(_db.genes.aggregate([
    {'$match': {'file': 'All_unfiltered_128k_multiexon_A0.75_minRead3_talon'}},
    {'$group': {'_id': '$gene.gene_name', 'gene_id': {'$push': '$gene_id_short'}}},
    {'$project': {'gene_name': '$_id', 'gene_id': '$gene_id', '_id': 0}}
]))
selectize = []
for x in g:
    if len(x['gene_id']) > 1:
        for id in x['gene_id']:
            selectize.append({
                'text': ' - '.join([x['gene_name'], id]),
                'value': id
            })
    else:
        selectize.append({
            'text': x['gene_name'],
            'value': x['gene_id'][0]
        })
selectize.sort(key=lambda x: x['text'])
#for x in selectize[:10]:
#    print(x)
#for x in selectize:
#    if ' - ' in x['text']:
#        print(x)

# abandoning this - let's stick with gene symbols

In [73]:
with open('all_symbols.js', 'w') as f:
    generate_selectize(f, 'All_unfiltered_128k_multiexon_A0.75_minRead3_talon')

Wrote 15672 gene symbols to all_symbols.js


In [102]:
def interval_union(intervals):
    """
    Returns the union of all intervals in the input list
      intervals: list of tuples or 2-element lists
    """
    intervals.sort(key=lambda x: x[0])
    union = [intervals[0]]
    for i in intervals[1:]:
        if i[0] <= union[-1][1]:  # overlap w/ previous
            if i[1] > union[-1][1]:  # only extend if larger
                union[-1][1] = i[1]
        else:
            union.append(i)
    return union

def mongo_bulk_write_model(ops, count=0):
    try:
        _db.model_exons.bulk_write(ops, ordered=False)
    except BulkWriteError as bwe:
        pprint(bwe.details)
    print('\twrote', count+len(ops), 'genes to db')
    return len(ops)

def mongo_collapse_transcripts(file1, files2=[]):
    genes = _db.genes.distinct('gene.gene_name', {'file': file1})
    filter = {'file': {'$in': files2}} if files2 else {}
    print(len(genes))
    ops = []
    count = 0
    for gene in genes:
        exon_coords = []
        model = []
        results = list(_db.genes.find(
            {'gene.gene_name': gene, **filter},
            {'gene.chromosome': 1, 'gene.strand': 1, 'gene.transcripts.transcript_type': 1, 'gene.transcripts.exons.start': 1, 'gene.transcripts.exons.end': 1}
        ))
        chrom = results[0]['gene']['chromosome']
        strand = results[0]['gene']['strand']
        for document in results:
            for transcript in document['gene']['transcripts']:
                if transcript.get('transcript_type') == 'retained_intron':
                    continue
                for exon in transcript['exons']:
                    exon_coords.append([exon['start'], exon['end']])
        new_coords = interval_union(exon_coords)
        #start_pos = np.min([i[0] for i in new_coords])
        #end_pos = np.max([i[1] for i in new_coords])
        if strand == '-':
            new_coords.reverse()
        for i, (start, end) in enumerate(new_coords, 1):
            model.append({
                'chromosome': chrom,
                'strand': strand,
                'start': start,
                'end': end,
                'exon_id': '_'.join([gene, str(i)]),
                'exon_number': i
            })
        ops.append(InsertOne({
            'gene_name': gene,
            'exons': model
        }))
        if len(ops) >= 1000:
            count += mongo_bulk_write_model(ops, count)
            ops = []
    if ops:
        mongo_bulk_write_model(ops, count)


In [111]:
mongo_collapse_transcripts('All_unfiltered_128k_multiexon_A0.75_minRead3_talon')

15672
	wrote 1000 genes to db
	wrote 2000 genes to db
	wrote 3000 genes to db
	wrote 4000 genes to db
	wrote 5000 genes to db
	wrote 6000 genes to db
	wrote 7000 genes to db
	wrote 8000 genes to db
	wrote 9000 genes to db
	wrote 10000 genes to db
	wrote 11000 genes to db
	wrote 12000 genes to db
	wrote 13000 genes to db
	wrote 14000 genes to db
	wrote 15000 genes to db
	wrote 15672 genes to db


In [99]:
_db.model_exons.create_index('gene_name', unique=True)
list(_db.model_exons.list_indexes())

[SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')]),
 SON([('v', 2), ('unique', True), ('key', SON([('gene_name', 1)])), ('name', 'gene_name_1')])]

In [ ]:
def mongo_add_expression(f, gene_file):
    tx = {}
    for i,line in enumerate(f):
        if i == 0:
            continue
        tokens = line.split()
        tx_id = tokens[3].replace('-', '_')
        cell_type = tokens[4]
        avg_exp = float(tokens[1])
        pct_exp = float(tokens[2])
        avg_exp_scaled = float(tokens[5])
        if tx_id not in tx:
            tx[tx_id] = {}
        tx[tx_id][cell_type] = [avg_exp, pct_exp, avg_exp_scaled]
    print(len(tx), 'total transcripts')
    write_count = 0
    other_count = 0
    no_match_count = 0
    for tx_id in tx:
        exp = [dict(zip(['cell_type', 'avg_exp', 'pct_exp', 'avg_exp_scaled'], [k]+v)) for k,v in tx[tx_id].items()]
        db_results = {x['file']: x['_id'] for x in _db.genes.find({'gene.transcripts.transcript_id': tx_id}, {'file': 1})}
        if gene_file in db_results:
            _db.genes.update_one(
                {'_id': db_results[gene_file], 'gene.transcripts.transcript_id': tx_id},
                {'$set': {'gene.transcripts.$.expression': exp}}
            )
            write_count += 1
        elif len(db_results) > 0:
            other_count += 1
        else:
            no_match_count += 1
        if write_count % 1000 == 0:
            print('wrote', write_count, 'transcripts to db')
    if write_count % 1000 != 0:
        print('wrote', write_count, 'transcripts to db')
    if other_count > 0:
        print(other_count, 'transcripts in db but not under', gene_file)
    if no_match_count > 0:
        print(no_match_count, 'transcripts not found in db')



In [31]:
with open('/mnt/z/Gandal/expression/Isoform_Average_percent_expression.txt') as f:
    mongo_add_expression(f, 'All_unfiltered_128k_multiexon_A0.75_minRead3_talon')

112134 total transcripts
wrote 1000 transcripts to db
wrote 2000 transcripts to db
wrote 3000 transcripts to db
wrote 4000 transcripts to db
wrote 5000 transcripts to db
wrote 6000 transcripts to db
wrote 7000 transcripts to db
wrote 8000 transcripts to db
wrote 9000 transcripts to db
wrote 10000 transcripts to db
wrote 11000 transcripts to db
wrote 12000 transcripts to db
wrote 13000 transcripts to db
wrote 13000 transcripts to db
wrote 14000 transcripts to db
wrote 15000 transcripts to db
wrote 16000 transcripts to db
wrote 17000 transcripts to db
wrote 18000 transcripts to db
wrote 19000 transcripts to db
wrote 20000 transcripts to db
wrote 21000 transcripts to db
wrote 22000 transcripts to db
wrote 23000 transcripts to db
wrote 24000 transcripts to db
wrote 25000 transcripts to db
wrote 26000 transcripts to db
wrote 27000 transcripts to db
wrote 28000 transcripts to db
wrote 29000 transcripts to db
wrote 29000 transcripts to db
wrote 30000 transcripts to db
wrote 31000 transcripts 